In [ ]:
import boto3
import time

In [ ]:
# ---------- Configurações ----------
REGION = "us-east-1"
INSTANCE_NAME = "Instancia-Teste-CloudWatch"
KEY_NAME = "sua-chave-ssh"
EMAIL = "seuemail@dominio.com"
SNS_TOPIC_NAME = "SNS-CloudWatch"
ALARM_NAME = "AlarmeCPU-Instancia"
BUCKET_NAME = "cloudtrail-logs-seunome-unico123"
TRAIL_NAME = "trilha-auditoria-seunome"
KMS_ALIAS = "alias/CloudTrailKey-seunome"

# ---------- Clientes boto3 ----------
ec2 = boto3.resource("ec2", region_name=REGION)
ec2_client = boto3.client("ec2", region_name=REGION)
cloudwatch = boto3.client("cloudwatch", region_name=REGION)
sns = boto3.client("sns", region_name=REGION)
s3 = boto3.client("s3", region_name=REGION)
cloudtrail = boto3.client("cloudtrail", region_name=REGION)
kms = boto3.client("kms", region_name=REGION)

# ---------- 1. Criar Instância EC2 ----------
print("Criando instância EC2...")
instance = ec2.create_instances(
    ImageId="ami-0c02fb55956c7d316",  # Amazon Linux 2 AMI (HVM), us-east-1
    InstanceType="t2.micro",
    KeyName=KEY_NAME,
    MinCount=1,
    MaxCount=1,
    SecurityGroups=["default"]
)[0]

instance.wait_until_running()
instance.reload()
instance_id = instance.id
print(f"Instância criada: {instance_id}")

# ---------- 2. Criar SNS Topic ----------
print("📡 Criando tópico SNS...")
topic = sns.create_topic(Name=SNS_TOPIC_NAME)
topic_arn = topic["TopicArn"]

sns.subscribe(
    TopicArn=topic_arn,
    Protocol="email",
    Endpoint=EMAIL
)
print(f"SNS criado: {topic_arn} (confirme no seu e-mail)")

# ---------- 3. Criar Alarme CloudWatch ----------
print("Criando alarme no CloudWatch...")
cloudwatch.put_metric_alarm(
    AlarmName=ALARM_NAME,
    ComparisonOperator="GreaterThanThreshold",
    EvaluationPeriods=1,
    MetricName="CPUUtilization",
    Namespace="AWS/EC2",
    Period=300,
    Statistic="Average",
    Threshold=70.0,
    ActionsEnabled=True,
    AlarmActions=[topic_arn],
    AlarmDescription="Alerta quando CPU > 70%",
    Dimensions=[{"Name": "InstanceId", "Value": instance_id}],
    Unit="Percent"
)
print("Alarme criado.")

# ---------- 4. Criar Bucket S3 ----------
print("🗄 Criando bucket S3 para logs CloudTrail...")
s3.create_bucket(
    Bucket=BUCKET_NAME,
    CreateBucketConfiguration={"LocationConstraint": REGION}
)
print(f"Bucket criado: {BUCKET_NAME}")

# ---------- 5. Criar KMS Key ----------
print("Criando chave KMS...")
key = kms.create_key(
    Description="Chave para criptografar CloudTrail",
    KeyUsage="ENCRYPT_DECRYPT",
    CustomerMasterKeySpec="SYMMETRIC_DEFAULT",
    Origin="AWS_KMS"
)
key_id = key["KeyMetadata"]["KeyId"]
kms.create_alias(AliasName=KMS_ALIAS, TargetKeyId=key_id)
print(f"KMS criado com alias {KMS_ALIAS}")

# ---------- 6. Criar CloudTrail ----------
print("Criando CloudTrail...")
cloudtrail.create_trail(
    Name=TRAIL_NAME,
    S3BucketName=BUCKET_NAME,
    KmsKeyId=key_id,
    IncludeGlobalServiceEvents=True,
    IsMultiRegionTrail=True
)
cloudtrail.start_logging(Name=TRAIL_NAME)
print(f"CloudTrail criado: {TRAIL_NAME}")

print("\n Ambiente configurado com sucesso!")
print("Confirme sua inscrição no SNS via e-mail para receber alertas.")
